<a href="https://colab.research.google.com/github/jeffvun/Machine-Learning-Labs/blob/main/LeukemiaClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###**Environment Setup**



<h3>  &nbsp;&nbsp;Using Colab Cloud TPU&nbsp;&nbsp; <a href="https://cloud.google.com/tpu/"><img valign="middle" src="https://raw.githubusercontent.com/GoogleCloudPlatform/tensorflow-without-a-phd/master/tensorflow-rl-pong/images/tpu-hexagon.png" width="25"></a></h3>

* Click Runtime and select **Change runtime type**. Set "TPU" as the hardware accelerator.
* The cell below makes sure you have access to a TPU on Colab.


In [ ]:
import os
assert os.environ['COLAB_TPU_ADDR']

The below cell will help install PyTorch, Torchvision, and PyTorch/XLA.

In [ ]:
!pip install cloud-tpu-client==0.10 torch==2.0.1 torchvision==0.15.2 https://storage.googleapis.com/tpu-pytorch/wheels/colab/torch_xla-2.0-cp310-cp310-linux_x86_64.whl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.9/162.9 MB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.2/120.2 kB 12.1 MB/s eta 0:00:00
  Attempting uninstall: uritemplate
    Found existing installation: uritemplate 4.1.1
    Uninstalling uritemplate-4.1.1:
      Successfully uninstalled uritemplate-4.1.1
  Attempting uninstall: google-api-core
    Found existing installation: google-api-core 2.11.1
    Uninstalling google-api-core-2.11.1:
      Successfully uninstalled google-api-core-2.11.1
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 2.84.0
    Uninstalling google-api-python-client-2.84.0:
      Successfully uninstalled google-api-python-client-2.84.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conf

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms
import torch_xla
import torch_xla.core.xla_model as xm

import matplotlib.pyplot as plt

from torch.utils.data import DataLoader, random_split
from torchvision.datasets import ImageFolder


In [ ]:
# Choosing gpu or cpu

def get_device():
    if torch.cuda.is_available():
        device = torch.device("cuda")
        print("Using GPU for computation.")
    elif 'TPU_NAME' in os.environ:
        device = xm.xla_device()
        print("Using TPU for computation.")
    else:
        device = torch.device("cpu")
        print("Using CPU for computation.")
    return device

device = get_device()

Using TPU for computation.


### **Data Preprocessing**

In [ ]:
# Defining the path to train and test data folders
train_folder = '/content/drive/MyDrive/Others/train'
test_folder = '/content/drive/MyDrive/Others/test'


In [ ]:
# Image transformations for data augmentation and normalization
transform = transforms.Compose([
    transforms.RandomResizedCrop(200),
    transforms.RandomHorizontalFlip(),
    transforms.GaussianBlur((5,9),(0.1,5)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])


In [ ]:
# Load the train and test datasets
train_dataset = ImageFolder(train_folder, transform=transform)
test_dataset = ImageFolder(test_folder, transform=transform)

print(f"Images in training data : {len(train_dataset)}")
print(f"Images in test data : {len(test_dataset)}")

Images in training data : 294
Images in test data : 97


### **Preparing Dataset**

In [ ]:
# Checking the size/structure of tensors we have created

img, label = train_dataset[0]
print(img.shape, label)

torch.Size([3, 200, 200]) 0


In [ ]:
# Checking for the number of classes

print("The following classes are there : \n",train_dataset.classes)

The following classes are there : 
 ['apples', 'tomatoes']


In [ ]:
# Spliting the train dataset into train and validation sets

train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])


In [ ]:
# Create data loaders for train, validation, and test datasets

batch_size = 128

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

### **Model Building**

Defining the CNN Architecture: 4 convolutional layers and an outer layer.


1.   Convolutional Layer Structure: Is a Convolutional Layer (Input image, Filter) with a ReLU activaton function and a MaxPooling Layer
2.   Fully Connected Layer: Has a Linear Activation function that takes on an Input vector of size N (size of the resized images) and gives an output of size K = 2 (2 classes : tomatoes and apples)



In [ ]:
# Step 2: Define the CNN model architecture

class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

        )
        self.fc_layer = nn.Sequential(
            nn.Flatten(),
            nn.Linear(512 * 6 * 6, 512), #size of input has to match nbr colums in matrix 1 (batchsize, inputsize)
            nn.ReLU(),
            nn.Linear(512,64),
            nn.ReLU(),
            nn.Linear(64, 2),
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)
        x = self.fc_layer(x)
        return x

    def training_step(self, batch):
        images, labels = batch[0].to(device), batch[1].to(device)
        out = self(images) # Generate predictions
        loss = nn.functional.cross_entropy(out, labels) # Calculate loss
        return loss

    def validation_step(self, batch):
        images, labels = batch[0].to(device), batch[1].to(device)
        out = self(images) # Generate predictions
        loss = nn.functional.cross_entropy(out, labels) # Calculate loss
        acc = accuracy(out, labels) # Calculate accuracy
        return {'val_loss': loss.detach(), 'val_acc': acc}

    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean() # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean() # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}

    def epoch_end(self, epoch, result):
        print("Epoch [{}], train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, result['train_loss'], result['val_loss'], result['val_acc']))

### **Hyper-parameter tuning**

In [ ]:
# Defining the accuracy, evaluation and fit methods

def accuracy(outputs, labels):
  _, preds = torch.max(outputs, dim=1)
  return torch.tensor(torch.sum(preds == labels).item() / len(preds))

@torch.no_grad()
def evaluate(model, val_loader):
  model.eval()
  outputs = [model.validation_step(batch) for batch in val_loader]
  return model.validation_epoch_end(outputs)

def fit(epochs, lr, model, train_loader, val_loader, opt_func = torch.optim.SGD):
    history = []
    optimizer = opt_func(model.parameters(),lr)
    for epoch in range(epochs):
        model.train()
        train_losses = []
        for batch in train_loader:
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        model.epoch_end(epoch, result)
        history.append(result)

    return history

In [ ]:
# model summary

model = CNNModel().to(device)
model

CNNModel(
  (conv_layers): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): ReLU()
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (12): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU()
    (14): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc_layer): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (

### **Model Training**

In [ ]:
# Train the model

num_epochs = 50
lr =0.001
opt_func = optim.Adam

history = fit(num_epochs, lr, model, train_loader, val_loader, opt_func)


Epoch [0], train_loss: 0.7262, val_loss: 0.6988, val_acc: 0.3729
Epoch [1], train_loss: 0.6945, val_loss: 0.6918, val_acc: 0.6271
Epoch [2], train_loss: 0.6920, val_loss: 0.6912, val_acc: 0.6102
Epoch [3], train_loss: 0.6889, val_loss: 0.6851, val_acc: 0.6102
Epoch [4], train_loss: 0.6789, val_loss: 0.6753, val_acc: 0.6102
Epoch [5], train_loss: 0.6633, val_loss: 0.6556, val_acc: 0.6271
Epoch [6], train_loss: 0.6395, val_loss: 0.6134, val_acc: 0.6271
Epoch [7], train_loss: 0.6039, val_loss: 0.6017, val_acc: 0.7288
Epoch [8], train_loss: 0.5810, val_loss: 0.6402, val_acc: 0.5932
Epoch [9], train_loss: 0.5466, val_loss: 0.6054, val_acc: 0.6780
Epoch [10], train_loss: 0.5436, val_loss: 0.7414, val_acc: 0.5424
Epoch [11], train_loss: 0.6060, val_loss: 0.6050, val_acc: 0.7288
Epoch [12], train_loss: 0.5656, val_loss: 0.6502, val_acc: 0.6102
Epoch [13], train_loss: 0.4996, val_loss: 0.6138, val_acc: 0.6949
Epoch [14], train_loss: 0.4725, val_loss: 0.6044, val_acc: 0.7288
Epoch [15], train_lo

RuntimeError: ignored

### **Model Evaluation**

In [ ]:
def plot_accuracies(history):
    print("Plot the history of accuracies")
    accuracies = [x['val_acc'] for x in history]
    plt.plot(accuracies, '-x')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.title('Accuracy vs. No. of epochs');


plot_accuracies(history)

In [ ]:
def plot_losses(history):
    print("Plot the losses in each epoch")
    train_losses = [x.get('train_loss') for x in history]
    val_losses = [x['val_loss'] for x in history]
    plt.plot(train_losses, '-bx')
    plt.plot(val_losses, '-rx')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(['Training', 'Validation'])
    plt.title('Loss vs. No. of epochs');

plot_losses(history)


In [ ]:
# Evaluate model on test data

model.eval()
correct, total = 0, 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {(100 * correct / total):.2f}%")

